In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt

csvphan= pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/phanns/allphan.csv', sep=',')
majt = csvphan
#majt= csvphan[((csvphan['Major capsid'] >= 8.50) | (csvphan['Major tail'] >= 8.50) | (csvphan['Minor capsid'] >= 8.50) | (csvphan['Baseplate'] >= 8.50) | (csvphan['Minor tail'] >= 8.50) | (csvphan['Portal'] >= 8.50) | (csvphan['Tail fiber'] >= 8.50) | (csvphan['Tail shaft'] >= 8.50) | (csvphan['Collar'] >= 8.50) | (csvphan['HTJ'] >= 8.50))& (csvphan['Confidence'] >= 0.85)]
#majt = majt[['seqname','Major capsid','Confidence']]
majt['seqname'] = majt['seqname'].apply(lambda x: x.split('_consensus')[0])
majt = majt.reset_index(drop=True)
majt = majt.sort_values('seqname')
csvfinal = majt.drop(columns= ['Other','Confidence'])
threshold = 8.50
csvfinal = csvfinal.apply(lambda x: x if x.name == 'seqname' else pd.to_numeric(x, errors='coerce').apply(lambda y: 0 if np.isnan(y) else (0 if y < threshold else 1)))
csvagrupado = csvfinal.groupby('seqname').agg({'Major capsid': 'sum', 'Major tail': 'sum','Minor capsid': 'sum', 'Baseplate': 'sum', 'Portal': 'sum', 'Tail fiber': 'sum','Tail shaft': 'sum', 'Collar': 'sum','HTJ': 'sum'})
capsidtotal = csvagrupado['Major capsid']+csvagrupado['Minor capsid']
tailtotal = csvagrupado['Major tail']+csvagrupado['Tail fiber']+csvagrupado['Tail shaft']
csvagrupado = csvagrupado.assign(Tail_proteins=tailtotal)
csvagrupado = csvagrupado.assign(Capsid_proteins=capsidtotal)
csvagrupado = csvagrupado.drop(columns=['Major capsid','Minor capsid','Major tail', 'Tail fiber', 'Tail shaft'])
csvagrupado.to_csv('agrupado4.csv')

#AREA DO HEATMAP

#Configs Heatmap
sns.set(rc={'figure.figsize':(128,9)})#deveria ser 9 (agora é 15) por 58 mas ficaria muito pequeno
sns.set(font_scale=2.0)
boundaries = np.arange(csvagrupado.values.min()-.5,csvagrupado.values.max()+1.5 )
ticks=np.arange(csvagrupado.values.min(),csvagrupado.values.max()+1 )
#Geração do heatmap e salvar figura #TRANSpose faz ficar horizontal, junto com a mudança de tamanho
heatmap = sns.heatmap(csvagrupado.transpose(),
                      cmap="viridis",
                      cbar_kws={"ticks":ticks, "boundaries":boundaries,"shrink":0.8})#vertical 0.3 fica bom
#heatmap = sns.heatmap(csvagrupado, cbar_kws={'label': 'Count','ticks': [0,1,2]}, cmap="tab20c")
heatfig = heatmap.get_figure()
heatfig.savefig('heatmap.svg')
#Depois de fagos completo fazer clustermap com protienas de cada grupo taxonomico

In [ ]:

#Rodei o codigo abaixo para os completos!

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt

csvagrupado= pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/phanns/allphan_completos.csv', sep=',', index_col=0)

#csvagrupado = csvagrupado.apply(pd.to_numeric, errors='coerce')

#AREA DO HEATMAP

#Configs Heatmap
sns.set(rc={'figure.figsize':(24,9)})#deveria ser 9 (agora é 15) por 58 mas ficaria muito pequeno
sns.set(font_scale=2.0)
boundaries = np.arange(float(csvagrupado.values.min()) - 0.5, float(csvagrupado.values.max()) + 1.5)
ticks = np.arange(float(csvagrupado.values.min()), float(csvagrupado.values.max()) + 1)
#Geração do heatmap e salvar figura #TRANSpose faz ficar horizontal, junto com a mudança de tamanho
heatmap = sns.heatmap(csvagrupado,
                      cmap="viridis",
                      cbar_kws={"shrink":0.3})#vertical 0.3 fica bom
#heatmap = sns.heatmap(csvagrupado, cbar_kws={'label': 'Count','ticks': [0,1,2]}, cmap="tab20c")
heatfig = heatmap.get_figure()
heatfig.savefig('heatmap.svg')
#Depois de fagos completo fazer clustermap com protienas de cada grupo taxonomico

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

data = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/phanns/allphan_completos.csv', sep=',', index_col=0)

plt.figure(figsize=(10, 8))  # Adjust the figure size as needed
sns.heatmap(data, annot=True, cmap='coolwarm', linewidths=.5)

# Show the heatmap
plt.show()

In [ ]:
#Trabalhar agora o CHECKV
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
checkv= pd.read_table(r'/content/drive/MyDrive/Colab Notebooks/phanns/quality_summary.tsv')
replace_non_alphanumeric = lambda col, val: re.sub('[^0-9a-zA-Z]+', '_', val) if col == 'contig_id' else val
checkv['contig_id'] = checkv.apply(lambda x: replace_non_alphanumeric('contig_id', x['contig_id']), axis=1)
checkv.head()
checkvtab1 = checkv[['contig_id','proviral_length']]
checkvtab1.plot.hist(grid=True, legend=False, figsize = (10,5))#Xlabel not working automatically -- Colocar em escala de cores
plt.xticks(fontsize=10)
plt.yticks (fontsize=10)
plt.title('Prophage genome lenght', size=14)
plt.xlabel('Sequence Length', size = 12)
plt.ylabel('Frequency', size=12)
plt.show()

In [ ]:
#Eggnog

Olhar dps https://pypi.org/project/pygenomeviz/

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

eggnog= pd.read_table(r'/content/drive/MyDrive/Colab Notebooks/phanns/eggnog_completos_v2.tsv')
eggnog = pd.DataFrame(eggnog)
eggnog2= eggnog[['COG_category']]
eggnog2 = eggnog2[eggnog2["COG_category"].str.contains("-") == False] #Retira os não anotados
category_counts = eggnog2.value_counts()
category_counts = eggnog2.value_counts().reset_index(name='count')
category_counts = pd.DataFrame(category_counts, columns=['COG_category', 'count'])
category_counts.set_index('COG_category', inplace=True)
category_counts.reset_index()
rows_less_than_6 = category_counts.loc[category_counts.sum(axis=1) < 10]
row_sum = rows_less_than_6.sum()
category_counts.loc['Other'] = row_sum
category_counts.drop(rows_less_than_6.index, inplace=True)
#print(category_counts)
category_counts.to_csv('cat_count.csv')
bar = category_counts.plot(kind='bar', y='count', figsize=(10,10), grid=True, legend=False)
plt.title('COG Distribution in propage-like elements', size=14)
plt.xlabel('COG Category', size = 12)
plt.ylabel('Quantity', size=12)
plt.xticks(fontsize=10)
plt.yticks (fontsize=10)
#pie.legend(loc='lower center', ncol= 6, bbox_to_anchor=(0.5,-0.5)) # Garante legenda abaixo do gráfico


# KEGG PATHWAYS

In [ ]:
import pandas as pd

# Carregue seus dados da tabela em um DataFrame do pandas
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/phanns/eggnog_completos_v2.tsv', sep= "\t")  # Substitua 'seu_arquivo.csv' pelo caminho para o seu arquivo

# Suponha que os nomes dos genes estejam em uma coluna chamada 'Gene'
data = data[['KEGG_Pathway']]
data = data[data["KEGG_Pathway"].str.contains("-") == False]
data['KEGG_Pathway'] = data['KEGG_Pathway'].str.split(',')
data = data.explode('KEGG_Pathway')
data = data[~data['KEGG_Pathway'].str.startswith('map')]
data.reset_index(drop=True, inplace=True)

# Use o método 'value_counts()' para calcular a frequência de cada gene
frequencia_genes = data['KEGG_Pathway'].value_counts()

# Crie um novo DataFrame com as colunas 'Gene' e 'Frequência'
df_frequencia_genes = pd.DataFrame({'Gene': frequencia_genes.index, 'Frequência': frequencia_genes.values})

# Exiba o novo DataFrame
print(df_frequencia_genes)


In [ ]:
import requests

# URL da API do KEGG para listar todos os Pathways
url = 'http://rest.kegg.jp/list/pathway/ko'

# Faça uma solicitação GET para obter a lista de Pathways
response = requests.get(url)

# Verifique se a solicitação foi bem-sucedida
if response.status_code == 200:
    # Divida a resposta em linhas
    lines = response.text.strip().split('\n')

    # Crie um dicionário para armazenar os códigos e nomes dos Pathways
    pathway_dict = {}

    # Itere pelas linhas e extraia os códigos e nomes
    for line in lines:
        parts = line.split('\t')
        if len(parts) == 2:
            code = parts[0].replace('path:', '')
            name = parts[1]
            pathway_dict[code] = name

    # Crie um DataFrame a partir do dicionário
    import pandas as pd
    pathway_df = pd.DataFrame(list(pathway_dict.items()), columns=['Código', 'Nome'])

    # Exiba o DataFrame
    print(pathway_df)
else:
    print('Erro ao acessar a API do KEGG')


In [ ]:
import pandas as pd

# Suponha que você já tenha os DataFrames df_frequencia_genes e pathway_df

# Crie um dicionário que mapeia códigos para nomes
codigo_para_nome = dict(zip(pathway_df['Código'], pathway_df['Nome']))

# Use a função map para substituir os códigos pelos nomes no DataFrame df_frequencia_genes
df_frequencia_genes['Gene'] = df_frequencia_genes['Gene'].map(codigo_para_nome)

# Exiba o DataFrame resultante
print(df_frequencia_genes)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Suponha que você já tenha o DataFrame df_frequencia_genes

# Ordene o DataFrame pela coluna 'Frequência' em ordem decrescente
df_frequencia_genes = df_frequencia_genes.sort_values(by='Frequência', ascending=False)

# Pegue os 10 genes mais comuns
top_10_genes = df_frequencia_genes.head(10)

# Crie um gráfico de barras para os 10 genes mais comuns
plt.figure(figsize=(12, 6))
plt.bar(top_10_genes['Gene'], top_10_genes['Frequência'])

# Rotacione os rótulos do eixo x para facilitar a leitura (opcional)
plt.xticks(rotation=90)

# Defina rótulos para os eixos
plt.xlabel('Vias metabólicas')
plt.ylabel('Frequência')

# Defina o título do gráfico
plt.title('Vias metabólicas mais encontradas')

# Exiba o gráfico
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd

# Carregue seus dados da tabela em um DataFrame do pandas
data = pd.read_csv('/content/drive/MyDrive/Sdd/eggnog.tsv', sep= "\t")  # Substitua 'seu_arquivo.csv' pelo caminho para o seu arquivo

# Suponha que os nomes dos genes estejam em uma coluna chamada 'Gene'
data = data[['KEGG_Pathway']]
data = data[data["KEGG_Pathway"].str.contains("-") == False]
data['KEGG_Pathway'] = data['KEGG_Pathway'].str.split(',')
data = data.explode('KEGG_Pathway')
data = data[~data['KEGG_Pathway'].str.startswith('ko')]
data.reset_index(drop=True, inplace=True)

# Use o método 'value_counts()' para calcular a frequência de cada gene
frequencia_genes = data['KEGG_Pathway'].value_counts()

# Crie um novo DataFrame com as colunas 'Gene' e 'Frequência'
df_frequencia_genes = pd.DataFrame({'Gene': frequencia_genes.index, 'Frequência': frequencia_genes.values})

# Exiba o novo DataFrame
print(df_frequencia_genes)


In [ ]:
import requests

# URL da API do KEGG para listar todos os Pathways
url = 'http://rest.kegg.jp/list/pathway/'

# Faça uma solicitação GET para obter a lista de Pathways
response = requests.get(url)

# Verifique se a solicitação foi bem-sucedida
if response.status_code == 200:
    # Divida a resposta em linhas
    lines = response.text.strip().split('\n')

    # Crie um dicionário para armazenar os códigos e nomes dos Pathways
    pathway_dict = {}

    # Itere pelas linhas e extraia os códigos e nomes
    for line in lines:
        parts = line.split('\t')
        if len(parts) == 2:
            code = parts[0].replace('path:', '')
            name = parts[1]
            pathway_dict[code] = name

    # Crie um DataFrame a partir do dicionário
    import pandas as pd
    pathway_df = pd.DataFrame(list(pathway_dict.items()), columns=['Código', 'Nome'])

    # Exiba o DataFrame
    print(pathway_df)
else:
    print('Erro ao acessar a API do KEGG')


In [ ]:
import pandas as pd

# Suponha que você já tenha os DataFrames df_frequencia_genes e pathway_df

# Crie um dicionário que mapeia códigos para nomes
codigo_para_nome = dict(zip(pathway_df['Código'], pathway_df['Nome']))

# Use a função map para substituir os códigos pelos nomes no DataFrame df_frequencia_genes
df_frequencia_genes['Gene'] = df_frequencia_genes['Gene'].map(codigo_para_nome)

# Exiba o DataFrame resultante
print(df_frequencia_genes)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Suponha que você já tenha o DataFrame df_frequencia_genes

# Ordene o DataFrame pela coluna 'Frequência' em ordem decrescente
df_frequencia_genes = df_frequencia_genes.sort_values(by='Frequência', ascending=False)

# Pegue os 10 genes mais comuns
top_10_genes = df_frequencia_genes.head(10)

# Crie um gráfico de barras para os 10 genes mais comuns
plt.figure(figsize=(12, 6))
plt.bar(top_10_genes['Gene'], top_10_genes['Frequência'])

# Rotacione os rótulos do eixo x para facilitar a leitura (opcional)
plt.xticks(rotation=90)

# Defina rótulos para os eixos
plt.xlabel('Vias metabólicas')
plt.ylabel('Frequência')

# Defina o título do gráfico
plt.title('Vias metabólicas mais encontradas')

# Exiba o gráfico
plt.tight_layout()
plt.show()


In [ ]:
df_frequencia_genes.to_csv('freq.csv')
pathway_df.to_csv('pathway.csv')

# KEGG PATHWAYS - MODULE


In [ ]:
import pandas as pd

# Carregue seus dados da tabela em um DataFrame do pandas
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/phanns/eggnog_completos_v2.tsv', sep= "\t")  # Substitua

# Suponha que os nomes dos genes estejam em uma coluna chamada 'Gene'
data = data[['KEGG_Pathway']]
data = data[data["KEGG_Pathway"].str.contains("-") == False]
data['KEGG_Pathway'] = data['KEGG_Pathway'].str.split(',')
data = data.explode('KEGG_Pathway')
data = data[~data['KEGG_Pathway'].str.startswith('map')]
data.reset_index(drop=True, inplace=True)

# Use o método 'value_counts()' para calcular a frequência de cada gene
frequencia_genes = data['KEGG_Pathway'].value_counts()

# Crie um novo DataFrame com as colunas 'Gene' e 'Frequência'
df_frequencia_genes = pd.DataFrame({'Gene': frequencia_genes.index, 'Frequência': frequencia_genes.values})

# Exiba o novo DataFrame
print(df_frequencia_genes)


In [ ]:
import requests

# URL da API do KEGG para listar todos os Pathways
url = 'http://rest.kegg.jp/list/pathway/ko'

# Faça uma solicitação GET para obter a lista de Pathways
response = requests.get(url)

# Verifique se a solicitação foi bem-sucedida
if response.status_code == 200:
    # Divida a resposta em linhas
    lines = response.text.strip().split('\n')

    # Crie um dicionário para armazenar os códigos e nomes dos Pathways
    pathway_dict = {}

    # Itere pelas linhas e extraia os códigos e nomes
    for line in lines:
        parts = line.split('\t')
        if len(parts) == 2:
            code = parts[0].replace('path:', '')
            name = parts[1]
            pathway_dict[code] = name

    # Crie um DataFrame a partir do dicionário
    import pandas as pd
    pathway_df = pd.DataFrame(list(pathway_dict.items()), columns=['Código', 'Nome'])

    # Exiba o DataFrame
    print(pathway_df)
else:
    print('Erro ao acessar a API do KEGG')


      Código                                              Nome
0    ko01100                                Metabolic pathways
1    ko01110             Biosynthesis of secondary metabolites
2    ko01120      Microbial metabolism in diverse environments
3    ko01200                                 Carbon metabolism
4    ko01210                   2-Oxocarboxylic acid metabolism
..       ...                                               ...
480  ko01503  Cationic antimicrobial peptide (CAMP) resistance
481  ko01521         EGFR tyrosine kinase inhibitor resistance
482  ko01524                          Platinum drug resistance
483  ko01523                             Antifolate resistance
484  ko01522                              Endocrine resistance

[485 rows x 2 columns]


In [ ]:
import pandas as pd

# Suponha que você já tenha os DataFrames df_frequencia_genes e pathway_df

# Crie um dicionário que mapeia códigos para nomes
codigo_para_nome = dict(zip(pathway_df['Código'], pathway_df['Nome']))

# Use a função map para substituir os códigos pelos nomes no DataFrame df_frequencia_genes
df_frequencia_genes['Gene'] = df_frequencia_genes['Gene'].map(codigo_para_nome)

# Exiba o DataFrame resultante
print(df_frequencia_genes)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Suponha que você já tenha o DataFrame df_frequencia_genes

# Ordene o DataFrame pela coluna 'Frequência' em ordem decrescente
df_frequencia_genes = df_frequencia_genes.sort_values(by='Frequência', ascending=False)

# Pegue os 10 genes mais comuns
top_10_genes = df_frequencia_genes.head(10)

# Crie um gráfico de barras para os 10 genes mais comuns
plt.figure(figsize=(12, 6))
plt.bar(top_10_genes['Gene'], top_10_genes['Frequência'])

# Rotacione os rótulos do eixo x para facilitar a leitura (opcional)
plt.xticks(rotation=90)

# Defina rótulos para os eixos
plt.xlabel('Vias metabólicas')
plt.ylabel('Frequência')

# Defina o título do gráfico
plt.title('Vias metabólicas mais encontradas')

# Exiba o gráfico
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd

# Carregue seus dados da tabela em um DataFrame do pandas
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/phanns/eggnog_completos_v2.tsv', sep= "\t")  # Substitua 'seu_arquivo.csv' pelo caminho para o seu arquivo

# Suponha que os nomes dos genes estejam em uma coluna chamada 'Gene'
data = data[['KEGG_Pathway']]
data = data[data["KEGG_Pathway"].str.contains("-") == False]
data['KEGG_Pathway'] = data['KEGG_Pathway'].str.split(',')
data = data.explode('KEGG_Pathway')
data = data[~data['KEGG_Pathway'].str.startswith('ko')]
data.reset_index(drop=True, inplace=True)

# Use o método 'value_counts()' para calcular a frequência de cada gene
frequencia_genes = data['KEGG_Pathway'].value_counts()

# Crie um novo DataFrame com as colunas 'Gene' e 'Frequência'
df_frequencia_genes = pd.DataFrame({'Gene': frequencia_genes.index, 'Frequência': frequencia_genes.values})

# Exiba o novo DataFrame
print(df_frequencia_genes)


In [ ]:
import requests

# URL da API do KEGG para listar todos os Pathways
url = 'http://rest.kegg.jp/list/pathway/'

# Faça uma solicitação GET para obter a lista de Pathways
response = requests.get(url)

# Verifique se a solicitação foi bem-sucedida
if response.status_code == 200:
    # Divida a resposta em linhas
    lines = response.text.strip().split('\n')

    # Crie um dicionário para armazenar os códigos e nomes dos Pathways
    pathway_dict = {}

    # Itere pelas linhas e extraia os códigos e nomes
    for line in lines:
        parts = line.split('\t')
        if len(parts) == 2:
            code = parts[0].replace('path:', '')
            name = parts[1]
            pathway_dict[code] = name

    # Crie um DataFrame a partir do dicionário
    import pandas as pd
    pathway_df = pd.DataFrame(list(pathway_dict.items()), columns=['Código', 'Nome'])

    # Exiba o DataFrame
    print(pathway_df)
else:
    print('Erro ao acessar a API do KEGG')


In [ ]:
import pandas as pd

# Suponha que você já tenha os DataFrames df_frequencia_genes e pathway_df

# Crie um dicionário que mapeia códigos para nomes
codigo_para_nome = dict(zip(pathway_df['Código'], pathway_df['Nome']))

# Use a função map para substituir os códigos pelos nomes no DataFrame df_frequencia_genes
df_frequencia_genes['Gene'] = df_frequencia_genes['Gene'].map(codigo_para_nome)

# Exiba o DataFrame resultante
print(df_frequencia_genes)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Suponha que você já tenha o DataFrame df_frequencia_genes

# Ordene o DataFrame pela coluna 'Frequência' em ordem decrescente
df_frequencia_genes = df_frequencia_genes.sort_values(by='Frequência', ascending=False)

# Pegue os 10 genes mais comuns
top_10_genes = df_frequencia_genes.head(10)

# Crie um gráfico de barras para os 10 genes mais comuns
plt.figure(figsize=(12, 6))
plt.bar(top_10_genes['Gene'], top_10_genes['Frequência'])

# Rotacione os rótulos do eixo x para facilitar a leitura (opcional)
plt.xticks(rotation=90)

# Defina rótulos para os eixos
plt.xlabel('Vias metabólicas')
plt.ylabel('Frequência')

# Defina o título do gráfico
plt.title('Vias metabólicas mais encontradas')

# Exiba o gráfico
plt.tight_layout()
plt.show()


In [ ]:
df_frequencia_genes.to_csv('freq.csv')
pathway_df.to_csv('pathway.csv')

Separa em classes, subclasses etc
Fazer anotação com genomas de plantas e um com virus
Genoma de referencia de bacteria --> Faz anotação dela
